In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [2]:
# Read all data files of one category.
import os
data_dir = 'data/JulianTrajs/alive'
trajectories = []  # dimensions: trajectory index -> time step -> coordinate (x, y, z)
for filename in os.listdir(data_dir):
    print filename, '->',
    trajectory = np.genfromtxt(os.path.join(data_dir, filename))
    print trajectory.shape[0], 'time steps'
    trajectories.append(trajectory)
trajectories = np.array(trajectories)

cell001.dat -> 577 time steps
cell002.dat -> 577 time steps
cell003.dat -> 577 time steps
cell004.dat -> 577 time steps
cell005.dat -> 577 time steps
cell006.dat -> 577 time steps
cell007.dat -> 577 time steps
cell008.dat -> 577 time steps
cell009.dat -> 577 time steps
cell010.dat -> 577 time steps
cell011.dat -> 577 time steps
cell012.dat -> 577 time steps
cell013.dat -> 577 time steps
cell014.dat -> 577 time steps
cell015.dat -> 577 time steps
cell016.dat -> 577 time steps
cell017.dat -> 577 time steps
cell018.dat -> 577 time steps
cell019.dat -> 577 time steps
cell020.dat -> 577 time steps
cell021.dat -> 577 time steps
cell022.dat -> 577 time steps
cell023.dat -> 577 time steps
cell024.dat -> 577 time steps
cell025.dat -> 577 time steps
cell026.dat -> 577 time steps
cell027.dat -> 577 time steps
cell028.dat -> 577 time steps
cell029.dat -> 577 time steps
cell030.dat -> 577 time steps
cell031.dat -> 577 time steps
cell032.dat -> 577 time steps
cell033.dat -> 577 time steps
cell034.da

In [3]:
# Normalize trajectories to [-1, 1] for the LSTM network (it outputs values between -1 and 1 by default).
# TODO: Try to change the activation in the LSTM to do this without normalization.
min_value = np.min(trajectories)
max_value = np.max(trajectories)
normalized_trajectories = np.interp(trajectories, [min_value, max_value], [-1., 1.])
np.min(normalized_trajectories), np.max(normalized_trajectories)

(-1.0, 0.99999999999999978)

In [4]:
# TODO: Split into training/test data here.
# TODO: Possible to use validation data in this scenario?


# Split trajectories into short parts following. Save the next position for each part.
# See also https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

maxlen = 20  # Change this to see how strong the current position depends on the positions long ago. REPHRASE THIS
step = 3  # Change this to vary redundancy. 
mini_trajectories = []
next_position = []

for trajectory in normalized_trajectories:
    for i in range(0, len(trajectory) - maxlen, step):
        mini_trajectories.append(trajectory[i:i+maxlen])
        next_position.append(trajectory[i+maxlen])
        
mini_trajectories = np.array(mini_trajectories)
next_position = np.array(next_position)

print 'Created', len(mini_trajectories), 'data samples'

Created 12090 data samples


In [5]:
mini_trajectories[0], next_position[0]

(array([[ 0.05523622,  0.05523622,  0.05523622],
        [ 0.05566071,  0.05779695,  0.05286426],
        [ 0.03838752,  0.06534696,  0.02709818],
        [ 0.03850322,  0.06699704,  0.02441499],
        [ 0.03880855,  0.06863345,  0.0219161 ],
        [ 0.03939917,  0.07057156,  0.01970224],
        [ 0.03924249,  0.07245225,  0.02243242],
        [ 0.03889297,  0.0738549 ,  0.02497102],
        [ 0.03828718,  0.06927952,  0.02155214],
        [ 0.03810676,  0.0606369 ,  0.02426429],
        [ 0.0379032 ,  0.06768063,  0.01555581],
        [ 0.03765616,  0.06326114,  0.01819746],
        [ 0.03749973,  0.05901885,  0.02092605],
        [ 0.03769094,  0.05532332,  0.01830577],
        [ 0.03821531,  0.05717232,  0.02172133],
        [ 0.04326945,  0.06417255,  0.02396337],
        [ 0.04289596,  0.06522634,  0.0207814 ],
        [ 0.04248285,  0.06104209,  0.02325294],
        [ 0.03706044,  0.05225908,  0.02641301],
        [ 0.04223672,  0.05350453,  0.01738139]]),
 array([ 0.0420370

In [6]:
# Set up the network.
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

model = Sequential()

# TODO: Try stateful for the same trajectory
# TODO: Play around with output_dim here. 
model.add(LSTM(input_shape=(maxlen, 3), output_dim=100, return_sequences=True))
model.add(LSTM(output_dim=3, return_sequences=False))
# TODO: Add Dropout and more LSTM layers. 

#model.add(Dropout(0.5))
#model.add(Dense(1))
#model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error', optimizer='rmsprop')

Using Theano backend.
D:\Python\27_32bit\lib\site-packages\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [7]:
# Train the model.

model.fit(mini_trajectories, next_position, nb_epoch=2, verbose=1, show_accuracy=True)

# TODO: See if GPU is better for RNN (according to keras docs)
# TODO: Play with batch_size
# TODO: Other implementation: learn on single trajectory, X = trajectory[:-1], y = trajectory[1:], then repeat this for the other trajectories
# Input and output dimension 3 in this case; does the network have to be stateful?
# TODO: maybe try displacement instead of absolute position

Epoch 1/2
12090/12090 [==============================] - 21s - loss: 0.0049 - acc: 0.8141    
Epoch 2/2
12090/12090 [==============================] - 27s - loss: 0.0013 - acc: 0.8805    


In [8]:
# Test the model predictions on the training data. 
predicted = np.interp(model.predict(mini_trajectories), [-1., 1.], [min_value, max_value])
expected = np.interp(next_position, [-1., 1.], [min_value, max_value])

In [9]:
zip(predicted[:10], expected[:10])

[(array([ -0.75435467,  -3.41881737, -16.68801743]),
  array([ -5.9775218,  -2.5539599, -18.50465  ])),
 (array([ -1.03532922,  -4.61531901, -19.08631518]),
  array([-11.508915 ,  -9.9777847, -14.942616 ])),
 (array([ -1.0471354 ,  -7.3686351 , -20.04031357]),
  array([-14.409026, -15.472929, -13.91908 ])),
 (array([ -1.58124432, -11.34438744, -20.9536812 ]),
  array([-16.458487, -24.46698 , -17.1917  ])),
 (array([ -2.27512974, -16.0746605 , -21.83129646]),
  array([-16.362498, -25.299648, -15.752455])),
 (array([ -3.59940567, -21.33231779, -22.70166235]),
  array([-17.900795, -31.346534, -10.781697])),
 (array([ -5.57029675, -27.18637897, -22.62566134]),
  array([-17.896284, -36.397257, -12.002794])),
 (array([ -7.48416457, -32.73457106, -21.8735987 ]),
  array([-17.946331, -38.934198, -13.284537])),
 (array([ -9.08585869, -37.89065693, -21.23006099]),
  array([-20.803707, -47.018157, -12.214188])),
 (array([-10.2753657 , -43.11573588, -20.8156153 ]),
  array([-22.121679, -50.310803,

In [10]:
# Rough measure for deviation.
np.sum(np.abs(predicted - expected)) / len(predicted) / 3.

8.3959524515767523

In [11]:
np.sum(np.abs(expected)) / len(expected) / 3.

43.062512918279715